In [1]:
import math
import numpy as np
import datetime as dt
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm
import pandas as pd
import cufflinks as cf

import Quandl as quandl

import plotly.plotly as py
import plotly.graph_objs as go
%matplotlib inline


In [7]:
pair =  ('AEP', 'LLY')

stock1 = 'WIKI/' + pair[0]
stock2 = 'WIKI/' + pair[1]

stocks = pd.concat([quandl.get(stock1, authtoken="1Cx13bkj4vDb7E13GLD6")['Adj. Close'].ix['01/01/2011':'04/30/2016'],
                    quandl.get(stock2, authtoken="1Cx13bkj4vDb7E13GLD6")['Adj. Close'].ix['01/01/2011':'04/30/2016']], 
                   axis=1).dropna()

stocks.columns = [pair[0], pair[1]]

stocks = stocks.apply(np.log10, axis=1)

In [ ]:
pair =  ('EWA', 'EWC')

stock1 = 'GOOG/AMEX_' + pair[0]
stock2 = 'GOOG/AMEX_' + pair[1]

stocks = pd.concat([quandl.get(stock1, authtoken="1Cx13bkj4vDb7E13GLD6")['Close'].ix['01/01/2011':'04/30/2016'],
                    quandl.get(stock2, authtoken="1Cx13bkj4vDb7E13GLD6")['Close'].ix['01/01/2011':'04/30/2016']], 
                   axis=1).dropna()

stocks.columns = [pair[0], pair[1]]

stocks = stocks.apply(np.log10, axis=1)

In [8]:
#conduct augmented dickey fuller or array x with a default
#level of 10%
def is_stationary(x, p):
    
    x = np.array(x)
    result = ts.adfuller(x, regression='ctt')
    #1% level
    if p == 1:
        #if DFStat <= critical value
        if result[0] >= result[4]['1%']:        #DFstat is less negative
            #is stationary
            return True
        else:
            #is nonstationary
            return False
    #5% level
    if p == 5:
        #if DFStat <= critical value
        if result[0] >= result[4]['5%']:        #DFstat is less negative
            #is stationary
            return True
        else:
            #is nonstationary
            return False
    #10% level
    if p == 10:
        #if DFStat <= critical value
        if result[0] >= result[4]['10%']:        #DFstat is less negative
            #is stationary
            return True
        else:
            #is nonstationary
            return False
    
    
#Engle-Granger test for cointegration for array x and array y
def are_cointegrated(x, y):

    #check x is I(1) via Augmented Dickey Fuller
    x_is_I1 = not(is_stationary(x, 10))
    #check y is I(1) via Augmented Dickey Fuller
    y_is_I1 = not(is_stationary(y, 10))
    #if x and y are no stationary        
    if x_is_I1 and y_is_I1:
        X = sm.add_constant(x)
        #regress x on y
        model = sm.OLS(np.array(y), np.array(X))
        results = model.fit()
        const = results.params[1]
        beta_1 = results.params[0]
        #solve for ut_hat
        u_hat = []
        for i in range(0, len(y)):
            u_hat.append(y[i] - x[i] * beta_1 - const)    
        #check ut_hat is I(0) via Augmented Dickey Fuller
        u_hat_is_I0 = is_stationary(u_hat, 1)
        #if ut_hat is I(0)
        if u_hat_is_I0:
            #x and y are cointegrated
            return True
        else:
            #x and y are not cointegrated
            return False 
    #if x or y are nonstationary they are not cointegrated
    else:
        return False
    
def pd_are_cointegrated(v):
    
    x = v.ix[:,0]
    y = v.ix[:,1]

    #check x is I(1) via Augmented Dickey Fuller
    x_is_I1 = not(is_stationary(x, 10))
    #check y is I(1) via Augmented Dickey Fuller
    y_is_I1 = not(is_stationary(y, 10))
    #if x and y are no stationary        
    if x_is_I1 and y_is_I1:
        X = sm.add_constant(x)
        #regress x on y
        model = sm.OLS(np.array(y), np.array(X))
        results = model.fit()
        const = results.params[1]
        beta_1 = results.params[0]
        #solve for ut_hat
        u_hat = []
        for i in range(0, len(y)):
            u_hat.append(y[i] - x[i] * beta_1 - const)    
        #check ut_hat is I(0) via Augmented Dickey Fuller
        u_hat_is_I0 = is_stationary(u_hat, 10)
        #if ut_hat is I(0)
        if u_hat_is_I0:
            #x and y are cointegrated
            return True
        else:
            #x and y are not cointegrated
            return False 
    #if x or y are nonstationary they are not cointegrated
    else:
        return False

In [39]:
#s = pd.concat([ (pd.Series(vwap(df.iloc[i:i+window]), index=[df.index[i+window]])) for i in range(len(df)-window) ])

In [9]:


window = 50

s = pd.concat([ (pd.Series(pd_are_cointegrated(stocks.iloc[i:i+window]), index=[stocks.index[i+window]])) 
              for i in range(len(stocks)-window)])



In [10]:
startlist = []
endlist = []


if s[0]:
    startlist.append(s.index[0])
    
test = False  
for date in s.index:
    if test:
        if not s[date]:
            test = False
            endlist.append(date)
    else: 
        if s[date]:
            test = True
            startlist.append(date)
            
if s[-1]:
    endlist.append(s.index[-1])
    
    
starttradelist = []
endtradelist = []


            
    

In [11]:


vspanlist = []
for i in range(0, len(startlist)):
    vspanlist.append({
            'x0': startlist[i],
            'x1': endlist[i], 
            'color': 'rgba(30,30,30,0.3)',
            'fill': True,
            'opacity': .4
        })

    
stocks.iplot(vspan=vspanlist, filename='Cointegration/Coint Example')